## SONAR: Emotionally Intelligent Music Recommendations

Draws heavily from [this tutorial](https://ahmedbesbes.com/sentiment-analysis-on-twitter-using-word2vec-and-keras.html)

Welcome to Sonar! This project was inpired me to first explore Machine Learning. The premise is simple: we often want different music to listen to based on our mood. However, oftentimes finding that music proves difficult. Sonar takes away that difficulty: by entering short mood messages, sonar generates Spotify recommendations using both your old favorites and determined mood. 

Enjoy!

In [56]:
import re

from bokeh.plotting import figure, output_notebook, show
from bokeh.models import HoverTool, BoxSelectTool

import gensim
from gensim.models.word2vec import Word2Vec
from keras import Sequential
from keras.layers import Dense
import numpy as np
from nltk.tokenize import TweetTokenizer
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
import spacy
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm_notebook, tqdm

nlp = spacy.load("en")
tqdm_notebook().pandas(desc="progress-bar")
tokenizer = TweetTokenizer(strip_handles=True, reduce_len=True)
TaggedDocument = gensim.models.doc2vec.TaggedDocument

### Preprocessing

- Load tweets into a usable format. Binarize sentiment from {0, 1, 2, 3, 4} to {0, 1} with 2 serving as the cut off for a "positive" sentiment. Clean up null values and reindex.

- Tokenize the data and take only the n values we aim to use in training.

In [3]:
def load_tweets():
    tweets_train = pd.read_csv("/home/malits/data/twitter_sentiment_analysis/training.1600000.processed.noemoticon.csv", 
                               encoding = "ISO-8859-1")
    data = pd.DataFrame()
    data["sentiment"] = tweets_train["0"].apply(lambda v: 1 if (v >= 2) else 0)
    data["sentiment"] = data["sentiment"].map(int)
    data["tweet"] = tweets_train.iloc[:, [5]]
    data["tweet"] = data["tweet"].astype(str)
    data = data[data['tweet'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index', axis=1, inplace=True)
    
    return data

In [4]:
data = load_tweets()
data.head()

,sentiment,tweet
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew


In [5]:
def tokenize(tweet):
    sub_regex = r'(#[A-Za-z0-9])|(https)\?!\.,'
    tweet = str(tweet)
    tokens = tokenizer.tokenize(tweet)
    tokens = [re.sub(sub_regex, '', t) for t in tokens]
    tokens = [t.lower() for t in tokens if t is not None and t != '']
    return tokens

In [18]:
def postprocess(data, n=1000000):
    data = data.head(n)
    data.loc[:, "tokens"] = data.loc[:, "tweet"].progress_apply(tokenize)
    data = data.loc[data.tokens != 'NC', :]
    data.reset_index(inplace=True)
    data.drop('index', inplace=True, axis=1)
    return data

In [19]:
processed = postprocess(data)

### Create training data

Once preprocessed, split to test/train sets and vectorize w w2vec for use in keras

In [20]:
# Size of our embedding space

N_DIM = 200

In [21]:
X_train, X_test, y_train, y_test = train_test_split(np.array(processed['tokens']),
                                                    np.array(processed['sentiment']),
                                                    test_size=0.2)

In [22]:
def labelizeTweets(tweets, label_type):
    labelized = []
    for i, v in tqdm_notebook(enumerate(tweets)):
        label = '{}_{}'.format(label_type, i)
        ## Use nltk TaggedDocument obj to mark words with their corresponding label. Makes words easier to retrieve from their vector form.
        labelized.append(TaggedDocument(v, [label]))
    return labelized

In [23]:
X_train = labelizeTweets(X_train, 'TRAIN')
X_test = labelizeTweets(X_test, 'TEST')

## Replace Word2Vec corpus with more data!

In [24]:
train_size = len(X_train)
tweet_w2v = Word2Vec(size=N_DIM, min_count=10)
tweet_w2v.build_vocab([x.words for x in tqdm_notebook(X_train)])
tweet_w2v.train([x.words for x in tqdm_notebook(X_train)], 
                total_examples=train_size, epochs = tweet_w2v.epochs)

(43251037, 60133050)

In [25]:
def get_similarity(text):
    return tweet_w2v.most_similar(text.lower())

In [26]:
get_similarity("Obama")

/home/malits/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('president', 0.656407356262207),
 ('martin', 0.6418095827102661),
 ('paul', 0.6215450763702393),
 ('steve', 0.6095056533813477),
 ('simon', 0.607050895690918),
 ('peter', 0.6052112579345703),
 ('jason', 0.5931947231292725),
 ('pres', 0.5923879146575928),
 ('dave', 0.5897916555404663),
 ('leader', 0.5890783071517944)]

## Transforming

We use TF-IDF weightings to weight word vectors accordingly

In [44]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x.words for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))

In [58]:
def create_tweet_vector(words):
    vecs = []
    
    for w in words:
        try:
            w = tweet_w2v.wv[w] * tfidf[w]
            vecs.append(w)
        except KeyError:
            continue
            
    return np.mean(vecs, axis=0)

In [59]:
train_vec_tweets = np.concatenate([create_tweet_vector(x, N_DIM) for x in tqdm_notebook(map(lambda x: x.words, X_train))])
train_vec_tweets = scale(train_vec_tweets)

test_vec_tweets = np.concatenate([create_tweet_vector(x, N_DIM) for x in tqdm_notebook(map(lambda x: x.words, X_test))])
test_vec_tweets = scale(test_vec_tweets)

TypeError: create_tweet_vector() takes 1 positional argument but 2 were given

## Neural Architecture in Keras

In [57]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=200))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.fit(matrix, y_test)

W0726 19:48:47.825564 140104529233728 deprecation_wrapper.py:119] From /home/malits/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0726 19:48:47.857306 140104529233728 deprecation_wrapper.py:119] From /home/malits/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0726 19:48:47.859565 140104529233728 deprecation_wrapper.py:119] From /home/malits/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0726 19:48:47.881644 140104529233728 deprecation_wrapper.py:119] From /home/malits/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0726 19

ValueError: Error when checking input: expected dense_1_input to have shape (200,) but got array with shape (23258,)